In [14]:
#    Only India JD
#    800 JDs after cleaning 
#    JDs from India


In [1]:
import pandas as pd
import string
import spacy 
import nltk

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth=-1

### Putting the data together

In [4]:
i1=pd.read_csv("indeed_in (1).csv", sep=",")
i2=pd.read_csv("indeed_in (2).csv", sep=",")
i3=pd.read_csv("indeed_in.csv", sep=",")


In [82]:
i1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2552 entries, 0 to 2551
Data columns (total 8 columns):
web-scraper-order        2552 non-null object
web-scraper-start-url    2552 non-null object
job_title                2552 non-null object
job_title-href           2552 non-null object
job_compay               638 non-null object
job_location             638 non-null object
job_description          638 non-null object
title                    638 non-null object
dtypes: object(8)
memory usage: 159.6+ KB


In [5]:
new=pd.concat([i1,i2,i3])
new=new.drop(['web-scraper-order', 'web-scraper-start-url', 'job_title-href',
       'title','job_compay', 'job_location'],axis=1)
new.columns


Index(['job_title', 'job_description'], dtype='object')

In [6]:
import chardet    
rawdata = open("ba_in_jobs.csv", 'rb').read()
result = chardet.detect(rawdata)
charenc = result['encoding']
print(charenc)

Windows-1252


In [7]:
i4=pd.read_csv("ba_in_jobs.csv",sep=",",encoding = 'Windows-1252')
i4=i4.drop(['company_name', 'job_location', 'ID'],axis=1)
print(i4.columns)
print(i4.shape)



Index(['job_title', 'job_description'], dtype='object')
(495, 2)


In [9]:
df=pd.concat([new,i4])
print(df.shape)



(3703, 2)


In [10]:
df=df[pd.isnull(df.job_description)==False]
df.shape

(1297, 2)

In [25]:
df.to_csv('ba_con.csv',header=True,index=False)

In [11]:
df['clean_text']=df['job_description'].apply(lambda x:x.translate(str.maketrans(string.punctuation,' '*len(string.punctuation))))

#Maketrans the first arguemenet is what needs to be replaced,the second arguement is what to be replaced with and the third arguement is the stuff u want to delete


df['clean_text']=df['clean_text'].apply(lambda x:x.translate(str.maketrans(string.digits,' '*len(string.digits))))

df['clean_text']=df['clean_text'].str.replace('\n',' ')
df['clean_text']=df['clean_text'].str.replace('\t',' ')
df['clean_text']=df['clean_text'].str.replace('\r',' ')

#df['clean_text']=df['clean_text'].str.lower()


#df['clean_text'].apply(lambda s:re.sub( r"([A-Z])", r" \1", s).split())#Splitting strings based on capital letters




In [14]:
import re

In [12]:
a=[]
def split_upper(dump):
    words = dump.split()
    for word in words:
        if len(word)>=10:
            a.append((' ').join(re.sub( r"([A-Z])", r" \1", word).split()))
        else:
            a.append(word)
    return (' ').join(a)

In [15]:
df['clean_text']=df['clean_text'].apply(lambda x:split_upper(x))

In [17]:
df.columns

Index(['job_title', 'job_description', 'clean_text'], dtype='object')

In [16]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\swarna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\swarna\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:
from nltk import word_tokenize, pos_tag, pos_tag_sents
from nltk.chunk import ne_chunk
nltk.download('maxent_ne_chunker')

import string


[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\swarna\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


In [36]:
#nouns = [token for token, pos in pos_tag(word_tokenize(sentence)) if pos.startswith('N','A','V')]

In [18]:
dic={"Power BI":"PowerBI","power BI":"PowerBI","DAX":"PowerBI","reporting":"Reporting_Analysis",
     "hoc reporting":"Reporting_Analysis","reporting analysis":"Reporting_Analysis","data mining":"DataMining",
     "data science":"DataScience","Data Science":"DataScience","data analysis":"DataAnalysis",
     "machine learning":"MachineLearning","Business Intelligence":"BusinessIntelligence",
     "spreadsheets":"Excel","Microsoft Office":"MicrosoftOffice","business intelligence":"BusinessIntelligence",
     "visualization":"visualisation","Dashboarding":"Dashboard"}

def replace_all(text,**dic):

    for i, j in dic.items():
        text = text.replace(i, j)
    return text

df['clean_text']=df['clean_text'].apply(lambda x:replace_all(x,**dic))


In [38]:
import numpy as np
#splitting df to avoid memory error
msk = np.random.rand(len(df)) < 0.30
train = df[msk]
test = df[~msk]
print(train.shape)

(376, 3)


In [78]:
df1, df2, df3,df4 = np.split(df.sample(frac=1), [int(.25*len(df)), int(.5*len(df)),int(0.8*len(df))])
print(df1.shape,df2.shape,df3.shape,df4.shape)

(324, 3) (324, 3) (389, 3) (260, 3)


In [39]:
from textblob import TextBlob
#from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
stop = stopwords.words('english')

def remove_stop(x):
     x = [w for w in x.split() if w not in set(stopwords)]  # remove stopwords
    return ' '.join(x)


In [83]:
df_list=[df2,df3,df4]

In [87]:
df2['text_stop']=df2['clean_text'].progress_apply(lambda cell:' '.join([item for item in cell.split() if item not in stop]))
    










  0%|                                                                                          | 0/324 [00:00<?, ?it/s]







  1%|▌                                                                                 | 2/324 [00:00<00:29, 11.05it/s]







  1%|▊                                                                                 | 3/324 [00:00<00:48,  6.62it/s]







  1%|█                                                                                 | 4/324 [00:00<00:56,  5.67it/s]







  2%|█▎                                                                                | 5/324 [00:01<01:41,  3.15it/s]







  2%|█▌                                                                                | 6/324 [00:01<02:10,  2.43it/s]







  2%|█▊                                                                                | 7/324 [00:02<02:17,  2.30it/s]







  2%|██                                                                                | 8/324 [00:02<0

MemoryError: 

In [22]:
import time
from tqdm import tqdm
tqdm.pandas()


C:\Users\swarna\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [42]:
for i in df_list:
    i['POS'] = pos_tag_sents( i['text_stop'].progress_apply(word_tokenize).tolist() )


 41%|████████████████████████████████▊                                               | 154/376 [00:54<02:52,  1.29it/s]


 58%|██████████████████████████████████████████████▍                                 | 218/376 [01:55<02:49,  1.07s/it]

MemoryError: 

In [ ]:
train['token_pos'] = train['POS'].progress_apply(lambda row:' '.join([i[0] for i in row if i[1] in ('JJ','JJR','JJS','RB','RBR','RBS','NNP','NN','NNS','NNPS','VB','VBD','VBG','VBN','VBZ','VBP')]))

In [ ]:
train['token_nouns'] = train['POS'].apply(lambda row:' '.join([i[0] for i in row if i[1] in ('JJ','JJR','JJS','NNP','NN','NNS','NNPS')]))

In [ ]:
# nlp = spacy.load('en_core_web_sm') 

In [ ]:
# tokens=[]
# #pos=[]
# #lemma=[]
# token_pos=[]
# token_noun=[]



# for doc in nlp.pipe(df['job_description'].astype('unicode').values,batch_size=500,n_threads=3):
#     if doc.is_parsed:   #is_parsed, bool, A flag indicating that the document has been syntactically parsed.
#         tokens.append([n.text for n in doc])
#         #pos.append([n.pos_ for n in doc])
#         token_pos.append(' '.join([n.text for n in doc if n.pos_ in ('NOUN','ADJ','VERB','ADV')]))
#         token_noun.append(' '.join([n.text for n in doc if n.pos_ in ('NOUN')]))

#         #lemma.append([n.lemma_ for n in doc])
        
#     else:
#         # We want to make sure that the lists of parsed results have the
#         # same number of entries of the original Dataframe, so add some blanks in case the parse fails
#         tokens.append(None)
#         #pos.append(None)
#         token_pos.append(None)
#         token_noun.append(None)

        

# df=df.assign(tokens=pd.Series(tokens))
# #df=df.assign(pos=pd.Series(pos))
# #df=df.assign(lemma=pd.Series(lemma))
# df=df.assign(token_pos=pd.Series(token_pos))
# df=df.assign(token_noun=pd.Series(token_noun))






In [ ]:
df[df.token_pos.isna()].shape

In [ ]:
df.columns

### Topic exploration

In [135]:
from nltk.corpus import stopwords
from gensim import corpora, models, similarities, matutils
from sklearn.feature_extraction.text import CountVectorizer

In [137]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)
# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans

In [143]:
df.columns

Index(['job_title', 'job_description', 'clean_text', 'text_stop',
       'tokenized_text', 'POS', 'Token_Nouns'],
      dtype='object')

In [214]:
corpus=df['Token_Nouns'].dropna().tolist()#list of positive reviews

#### Latent Semantic Analysis

In [226]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [246]:
tf_idf = TfidfVectorizer(stop_words=my_stop_words,ngram_range=(1, 3))
tfidf_vec = tf_idf.fit_transform(corpus)
#idf_df = pd.DataFrame(tfidf_vec.toarray(), columns=tf_idf.get_feature_names())


In [264]:
lsa = TruncatedSVD(8, algorithm="arpack" )
lsa_vecs = lsa.fit_transform(tfidf_vec)
lsa_vecs = Normalizer(copy=False).fit_transform(lsa_vecs)
feature_names = tf_idf.get_feature_names()
#lsa_df = pd.DataFrame(lsa.components_.round(5), columns=feature_names)

In [265]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

no_top_words = 25
display_topics(lsa, feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights,Topic 5 words,Topic 5 weights,Topic 6 words,Topic 6 weights,Topic 7 words,Topic 7 weights
0,data,0.3,requirements,0.1,requirements,0.1,data,0.2,seo,0.2,business,0.1,big data,0.1,sales,0.2
1,using,0.2,work,0.1,ensure requirements,0.1,crm,0.1,marketing,0.2,analytics,0.1,big,0.1,data analysis statistics,0.1
2,learning,0.1,business,0.1,capturing,0.1,founder,0.1,search,0.2,data,0.1,big data machine,0.1,analysis statistics finance,0.1
3,marketing,0.1,data,0.1,met,0.1,data architect,0.1,time job,0.1,sales,0.1,data management advanced,0.1,statistics finance business,0.1
4,models,0.1,analytics,0.1,ensure,0.1,years required,0.1,search engine,0.1,technology,0.1,transitioned clients,0.1,statistics finance,0.1
5,experience,0.1,development,0.1,gathering,0.1,architect,0.1,social media marketing,0.1,management,0.1,mining bi,0.1,analysis statistics,0.1
6,sales,0.1,innovation,0.1,development,0.1,vcs,0.1,engine optimization,0.1,team,0.1,experiencetredence analytics,0.1,sales data analysis,0.1
7,business,0.1,ensure requirements,0.1,software development,0.1,data architecture,0.1,search engine optimization,0.1,analysis,0.1,experiencetredence,0.1,shift,0.1
8,machine learning,0.1,care,0.1,business requirements,0.1,mysql,0.1,media marketing,0.1,experience,0.1,learning bringing,0.1,finance business,0.1
9,machine,0.1,capturing,0.1,document,0.0,backed,0.1,time,0.1,solutions,0.1,existing technology data,0.1,status,0.1


In [210]:
from sklearn.feature_extraction import text

my_stop_words = text.ENGLISH_STOP_WORDS.union(["concentration","apply","proven","bangalore","mumbai","karnataka","maharashtra"])


#### Document clustering with Kmeans

In [197]:
from sklearn.cluster import KMeans

num_clusters = 10 #Change it according to your data.
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_vec)
clusters = km.labels_.tolist()



idea={'Idea':corpus, 'Cluster':clusters} #Creating dict having doc with the corresponding cluster number.
frame=pd.DataFrame(idea,index=[clusters], columns=['Idea','Cluster']) # Converting it into a dataframe.


#frame['Cluster'].value_counts()




In [198]:
frame['Cluster'].value_counts()


0    756
1    114
7    75 
8    60 
2    60 
9    58 
6    58 
3    45 
5    43 
4    28 
Name: Cluster, dtype: int64

In [203]:
print("Top terms per cluster:",'\n')
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tf_idf.get_feature_names()
for i in range(10):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :100]]
    print("Cluster {}: {}{}".format(i, ' '.join(top_ten_words),'\n'))

Top terms per cluster: 

Cluster 0: sales marketing design project performance problems products reporting software processes engineering people environment operations research reports supply chain planning supply chain client clients risk ensure level status quality building dashboards strategy service position market finance systems india manage testing excel business intelligence impact banking platform degree analyst manager bi organization ms trust inventory technologies end leading enterprise making model amazon bengaluru program strong techniques define etl northern trust northern deliver tableau com decision understand delivery methods candidate highly communication skills needs related members applications target expertise cloud metrics practices lead application opportunity areas effectively driving managing results skills ability day focus goals hands great description

Cluster 1: marketing sales experience using regression machine learning machine ds sales marketing data se


Cluster 6: innovation care insight trade consumer analytics consumer receive training media trade retailers data analytics business analysts receive predictive valued today ba based data brands environment analysts diversity media core training life day initiatives ideas market data science value ability query analyze trade efficiency ability query data mining ai opportunities brands trade cost inclusion valued algorithms conclusions algorithms conclusions business trade cost saving channels retailers trade efficiency media capacity crafting tomorrow capacity crafting growth opportunities brands channels retailers levels crafting mentorship collaborate mentorship collaborate passionate trade channels retailers crafting tomorrow innovation innovation passion hardworking crafting tomorrow innovation passion inclusion valued included sciencespreferred trade channels business analysts deliver data market share environment place work heard inspired heard inspired offer environment place bu

### Key word extraction using word2vec

In [32]:
import multiprocessing

from gensim.models import Word2Vec
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis

from matplotlib import pyplot

In [33]:
import re

In [34]:
df.columns

Index(['job_title', 'job_description', 'clean_text', 'text_stop'], dtype='object')

In [35]:
# Create the list of list format of the custom corpus for gensim modeling 
sent = [row.split(' ') for row in df['token_pos'].dropna()]
# show the example of list of list format of the custom corpus for gensim mode
len(sent)

KeyError: 'token_pos'

In [ ]:
# Iterate over the list and add the size of all internal lists 
count = 0
for listElem in sent:
    count += len(listElem)                    
 
print('Total Number of elements : ', count)

In [ ]:
model = Word2Vec(sent, min_count=1,size= 220,workers=5, window =5, sg = 1)

X = model[model.wv.vocab]

pca = PCA(n_components=3)
result = pca.fit_transform(X)
tmp = pd.DataFrame(result, index=model.wv.vocab, columns=['x', 'y','z'])


In [ ]:
# converts the word2vec term matrix into a datafarame
ordered_vocab = [(term, voc.index, voc.count) for term, voc in model.wv.vocab.items()]
ordered_vocab = sorted(ordered_vocab, key=lambda k: k[2])
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)
word_vectors = pd.DataFrame(model.wv.syn0[term_indices, :], index=ordered_terms)


In [ ]:
hard_skills=['Excel', 'SPSS', 'SAS', 'SQL', 'Oracle', 'PowerBI', 'Qlik', 'Python', 'R', 'Tableau', 
             'Qualtrics', 'SAP', 'Pentaho', 'Shiny','Pandas', 'Alteryx', 'Oracle', 'Cognos', 'Hive',
              'Pyspark','HDFS','MapReduce','Hadoop','ETL','KPI']

concept_skills=['forecasting','MachineLearning', 'DataMining', 'DataScience', 'metrics', 'statistical', 
                'predictive', 'DataAnalysis', 'BusinessIntelligence',  'visualisation', 'dashboards', 
                'survey', 'database','regression','Reporting_Analysis']

In [ ]:
tools=[]
concepts=[]

for item in hard_skills:
    tools.append(model.wv.most_similar(positive=[item],topn=30))
    
for item in concept_skills:
    concepts.append(model.wv.most_similar(positive=[item],topn=30))
    
    


In [ ]:
print(concepts)

In [ ]:
tools_name= [i[0] for item in tools for i in item ]
print('No of tools',len(tools_name))

tools_corr= [i[1] for item in tools for i in item ]
print('No of tools',len(tools_name))

concept_name= [i[0] for item in concepts for i in item ]
print('No of tools',len(concept_name))

concept_corr= [i[1] for item in concepts for i in item ]
print('No of tools',len(concept_name))

In [38]:
tools_tuples = list(zip(tools_name,tools_corr))
tools=pd.DataFrame(data_tuples, columns=['Tools','Corr'])
print(tools.shape)

concept_tuples = list(zip(concept_name,concept_corr))
concepts=pd.DataFrame(concept_tuples, columns=['concepts','Corr'])
print(concepts.shape)

NameError: name 'tools_name' is not defined

In [210]:
tools.to_csv('tools.csv',header=True)
concepts.to_csv('concepts.csv',header=True)



In [36]:
import qgrid

In [37]:
#to show a df simply use the below:
qgrid.show_grid(tools)

NameError: name 'tools' is not defined